# Using Convolutional Neural Network (CNN)

### Import the Required Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from PIL import Image as PILImage

### Loading the MNIST dataset for model

In [2]:
# Load data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

### Preprocessing the dataset

In [3]:
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

### Splitting the traning data
splitting the traning dataset into traning and validation set

In [4]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

### Defining my CNN model

In [5]:
# Define and compile the CNN model
cnn_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.25),
    layers.Dense(10, activation='softmax')
])

C:\Users\umerj\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Compiling the CNN model

In [6]:
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Model Training Callbacks

In [7]:
# Define callbacks
es = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=5, verbose=1, restore_best_weights=True)
mc = ModelCheckpoint('cnn_model_best.keras', monitor='val_accuracy', verbose=1, save_best_only=True)
cb = [es, mc]

### Data Generators for Augmentation and Validation

In [8]:
# Create data generators
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    vertical_flip=False
)
val_datagen = ImageDataGenerator()


### Data Generators for Training and Validation

In [9]:
# Flow data from arrays
train_generator = train_datagen.flow(x_train, y_train, batch_size=32)
val_generator = val_datagen.flow(x_val, y_val, batch_size=32)

### Training CNN Model with Data Augmentation

In [ ]:
# Train the CNN model with data augmentation
history_cnn = cnn_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=cb,
    verbose=1
)


Epoch 1/10
   3/1313 ━━━━━━━━━━━━━━━━━━━━ 36s 28ms/step - accuracy: 0.0868 - loss: 2.3006

C:\Users\umerj\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1313/1313 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.7511 - loss: 0.7747
Epoch 1: val_accuracy improved from -inf to 0.97206, saving model to cnn_model_best.keras
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 51s 37ms/step - accuracy: 0.7512 - loss: 0.7745 - val_accuracy: 0.9721 - val_loss: 0.0945
Epoch 2/10
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.9360 - loss: 0.2046
Epoch 2: val_accuracy improved from 0.97206 to 0.98061, saving model to cnn_model_best.keras
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 43s 32ms/step - accuracy: 0.9360 - loss: 0.2046 - val_accuracy: 0.9806 - val_loss: 0.0642
Epoch 3/10
1311/1313 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.9530 - loss: 0.1546
Epoch 3: val_accuracy improved from 0.98061 to 0.98167, saving model to cnn_model_best.keras
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 38s 29ms/step - accuracy: 0.9530 - loss: 0.1546 - val_accuracy: 0.9817 - val_loss: 0.0592
Epoch 4/10
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9631 - loss: 0.1223
Epoch 4: val_acc

### Evaluating CNN model

In [ ]:
# Evaluate the CNN model
cnn_model = models.load_model('cnn_model_best.keras')
cnn_test_loss, cnn_test_acc = cnn_model.evaluate(x_test, y_test)
print('CNN Test accuracy:', cnn_test_acc)

### Making Predictions on test data

In [ ]:
# Use trained CNN model to predict on test data
cnn_predictions = cnn_model.predict(x_test)
cnn_pred_labels = np.argmax(cnn_predictions, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

### Counting occurance of digits in dataset

In [ ]:
# Count occurrences of each digit in the dataset
unique, counts = np.unique(y_test_labels, return_counts=True)
digit_counts = dict(zip(unique, counts))

# Print digit counts
print("Digit counts in test dataset:")
for digit, count in digit_counts.items():
    print(f"Digit {digit}: {count} occurrences")

# Draw bar graph for digit counts
plt.figure(figsize=(10, 6))
plt.bar(digit_counts.keys(), digit_counts.values(), color='skyblue')
plt.xlabel('Digit')
plt.ylabel('Occurrences')
plt.title('Digit Counts in Test Dataset')
plt.xticks(range(10))
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

### Displaying model summary

In [ ]:
# Model summary for CNN
print("\nCNN Model Summary:")
cnn_model.summary()


### Plot CNN training & validation accuracy values


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(history_cnn.history['accuracy'])
plt.plot(history_cnn.history['val_accuracy'])
plt.title('CNN Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])
plt.show()

### Generate and plot CNN confusion matrix

In [ ]:

conf_matrix_cnn = confusion_matrix(y_test_labels, cnn_pred_labels)
conf_matrix_cnn_display = ConfusionMatrixDisplay(conf_matrix_cnn, display_labels=range(10))
conf_matrix_cnn_display.plot(cmap=plt.cm.Blues)
plt.title('CNN Confusion Matrix')
plt.show()

### R2 Score

In [ ]:
# Evaluate CNN model with R2 score
cnn_r2 = r2_score(y_test_labels, cnn_pred_labels)
print('CNN R2 Score:', cnn_r2)

### Sample Predictions

In [ ]:

plt.figure(figsize=(12, 12))
for i in range(16):  # Display 16 samples
    plt.subplot(4, 4, i + 1)
    plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
    plt.title(f'Pred: {cnn_pred_labels[i]}\nAct: {y_test_labels[i]}')
    plt.axis('off')
plt.tight_layout()
plt.show()

# Random Forest

### Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, r2_score
import joblib
import random

### Prepare data for Random Forest

In [ ]:
# Prepare data for Random Forest
x_train_flat = x_train.reshape(-1, 28 * 28)
x_test_flat = x_test.reshape(-1, 28 * 28)


### Datapreprocessing 

In [ ]:
# Normalize pixel values
scaler = StandardScaler()
x_train_flat = scaler.fit_transform(x_train_flat)
x_test_flat = scaler.transform(x_test_flat)

### Traning Random Forest model

In [ ]:
# Train Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train_flat, np.argmax(y_train, axis=1))

### Making Predictions

In [ ]:
# Predict with Random Forest
rf_predictions = rf_model.predict(x_test_flat)
rf_pred_labels = rf_predictions

### Confusion Matrix

In [ ]:
# Generate and plot Random Forest confusion matrix
conf_matrix_rf = confusion_matrix(np.argmax(y_test, axis=1), rf_pred_labels)
conf_matrix_rf_display = ConfusionMatrixDisplay(conf_matrix_rf, display_labels=range(10))
conf_matrix_rf_display.plot(cmap=plt.cm.Blues)
plt.title('Random Forest Confusion Matrix')
plt.show()

### Evaluating Model

In [ ]:
# Evaluate Random Forest model
rf_accuracy = np.mean(rf_pred_labels == np.argmax(y_test, axis=1))
print('Random Forest Test accuracy:', rf_accuracy)


### R2 Score

In [ ]:
# Evaluate Random Forest model with R2 score
rf_r2 = r2_score(np.argmax(y_test, axis=1), rf_pred_labels)
print('Random Forest R2 Score:', rf_r2)


### Making Single Prediction

In [ ]:
# Single prediction on a random image from test set
random_idx = random.randint(0, len(x_test) - 1)
random_image = x_test[random_idx].reshape(1, 28 * 28)  # Flatten for Random Forest

# Predict with Random Forest
rf_single_pred = rf_model.predict(random_image)
rf_single_pred_label = rf_single_pred[0]

print(f'Random Forest Single Prediction: {rf_single_pred_label}'). in this display image too

### Saving the Model

In [ ]:
# Save the Random Forest model and scaler
joblib.dump(rf_model, 'random_forest_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
print("Random Forest model and scaler saved to 'random_forest_model.pkl' and 'scaler.pkl'")